In [3]:
import zipfile
import os

# Paths to the uploaded zip files
mit_bih_path = 'M:\\Dissertation\\mit-bih-arrhythmia-database-1.0.0.zip'
noise_test_path = 'M:\\Dissertation\\mit-bih-noise-stress-test-database-1.0.0.zip'
extract_dir = 'M:\\Dissertation\\New folder'

# Function to unzip the files
def unzip_file(zip_path, extract_to):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)

# Unzip the files
unzip_file(mit_bih_path, extract_dir)
unzip_file(noise_test_path, extract_dir)

# List the extracted files to verify
os.listdir('M:\\Dissertation\\New folder\\mit-bih-arrhythmia-database-1.0.0'), os.listdir('M:\\Dissertation\\New folder\\mit-bih-noise-stress-test-database-1.0.0')


(['100.atr',
  '100.dat',
  '100.hea',
  '100.xws',
  '101.atr',
  '101.dat',
  '101.hea',
  '101.xws',
  '102-0.atr',
  '102.atr',
  '102.dat',
  '102.hea',
  '102.xws',
  '103.atr',
  '103.dat',
  '103.hea',
  '103.xws',
  '104.atr',
  '104.dat',
  '104.hea',
  '104.xws',
  '105.atr',
  '105.dat',
  '105.hea',
  '105.xws',
  '106.atr',
  '106.dat',
  '106.hea',
  '106.xws',
  '107.atr',
  '107.dat',
  '107.hea',
  '107.xws',
  '108.atr',
  '108.at_',
  '108.dat',
  '108.hea',
  '108.xws',
  '109.atr',
  '109.dat',
  '109.hea',
  '109.xws',
  '111.atr',
  '111.dat',
  '111.hea',
  '111.xws',
  '112.atr',
  '112.dat',
  '112.hea',
  '112.xws',
  '113.atr',
  '113.dat',
  '113.hea',
  '113.xws',
  '114.atr',
  '114.dat',
  '114.hea',
  '114.xws',
  '115.atr',
  '115.dat',
  '115.hea',
  '115.xws',
  '116.atr',
  '116.dat',
  '116.hea',
  '116.xws',
  '117.atr',
  '117.at_',
  '117.dat',
  '117.hea',
  '117.xws',
  '118.atr',
  '118.dat',
  '118.hea',
  '118.xws',
  '119.atr',
  '119.at_

In [ ]:
649984

In [1]:
import numpy as np
import wfdb
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import os

class ECGDataset(Dataset):
    def __init__(self, raw_signals, noisy_signals):
        self.raw_signals = raw_signals
        self.noisy_signals = noisy_signals

    def __len__(self):
        return len(self.raw_signals)

    def __getitem__(self, idx):
        raw_signal = self.raw_signals[idx]
        noisy_signal = self.noisy_signals[idx]
        return torch.tensor(raw_signal, dtype=torch.float32), torch.tensor(noisy_signal, dtype=torch.float32)

def load_mit_bih_data(records, target_length=650000):
    raw_signals = []
    noisy_signals = []
    
    for record in records:
        raw_record = wfdb.rdrecord(f'M:\\Dissertation\\New folder\\mit-bih-arrhythmia-database-1.0.0/{record}')
        raw_signal = raw_record.p_signal[:, 0]  # Use the first channel for simplicity
        
        # Load noise and add it to the raw signal
        noise_record = wfdb.rdrecord(f'M:\\Dissertation\\New folder\\mit-bih-noise-stress-test-database-1.0.0/em')  # Example for EM noise
        noise_signal = noise_record.p_signal[:, 0]
        
        # Ensure the signals are of the same length
        min_length = min(len(raw_signal), len(noise_signal), target_length)
        raw_signal = raw_signal[:min_length]
        noise_signal = noise_signal[:min_length]
        noisy_signal = raw_signal + noise_signal
        
        # Pad signals to target length
        if min_length < target_length:
            raw_signal = np.pad(raw_signal, (0, target_length - min_length), 'constant')
            noisy_signal = np.pad(noisy_signal, (0, target_length - min_length), 'constant')
        
        raw_signals.append(raw_signal)
        noisy_signals.append(noisy_signal)
    
    return np.array(raw_signals), np.array(noisy_signals)

# Select 10 records for the experiment
records = ['103', '105', '111', '116', '122', '205','213', '219', '223', '230']
target_length = 649984
raw_signals, noisy_signals = load_mit_bih_data(records, target_length=target_length)

# Create dataset and dataloader
dataset = ECGDataset(raw_signals, noisy_signals)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)


In [2]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv1d(1, 16, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv1d(16, 32, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv1d(32, 64, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv1d(64, 128, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv1d(128, 256, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv1d(256, 512, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv1d(512, 1024, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True)
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose1d(1024, 512, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.ConvTranspose1d(512, 256, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.ConvTranspose1d(256, 128, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.ConvTranspose1d(128, 64, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.ConvTranspose1d(64, 32, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.ConvTranspose1d(32, 16, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.ConvTranspose1d(16, 1, kernel_size=4, stride=2, padding=1),
            nn.Tanh()
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            nn.Conv1d(2, 64, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv1d(64, 128, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv1d(128, 256, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv1d(256, 512, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv1d(512, 1, kernel_size=4, stride=2, padding=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.main(x).view(x.size(0), -1)

def train(generator, discriminator, dataloader, num_epochs=5, lr=0.0002):
    criterion = nn.BCELoss()
    optimizer_G = torch.optim.Adam(generator.parameters(), lr=lr, betas=(0.5, 0.999))
    optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=lr, betas=(0.5, 0.999))

    for epoch in range(num_epochs):
        for i, (raw_signals, noisy_signals) in enumerate(dataloader):
            batch_size = raw_signals.size(0)
            
            # Ensure the signals have the same length
            min_length = min(raw_signals.shape[-1], noisy_signals.shape[-1])
            raw_signals = raw_signals[:, :min_length]
            noisy_signals = noisy_signals[:, :min_length]

            # Denoise the noisy signals
            noisy_signals = noisy_signals.unsqueeze(1)  # Add channel dimension
            raw_signals = raw_signals.unsqueeze(1)

            # Train Generator
            optimizer_G.zero_grad()
            gen_signals = generator(noisy_signals)
            
            # Update valid and fake labels to match the discriminator output size
            disc_output_size = discriminator(torch.cat((gen_signals, noisy_signals), 1)).size()
            valid = torch.ones(disc_output_size).to(gen_signals.device)
            fake = torch.zeros(disc_output_size).to(gen_signals.device)
            
            g_loss = criterion(discriminator(torch.cat((gen_signals, noisy_signals), 1)), valid)
            g_loss.backward()
            optimizer_G.step()

            # Train Discriminator
            optimizer_D.zero_grad()
            real_loss = criterion(discriminator(torch.cat((raw_signals, noisy_signals), 1)), valid)
            fake_loss = criterion(discriminator(torch.cat((gen_signals.detach(), noisy_signals), 1)), fake)
            d_loss = (real_loss + fake_loss) / 2
            d_loss.backward()
            optimizer_D.step()

            print(f"[Epoch {epoch}/{num_epochs}] [Batch {i}/{len(dataloader)}] [D loss: {d_loss.item()}] [G loss: {g_loss.item()}]")

# Initialize models
generator = Generator()
discriminator = Discriminator()

# Train the models
train(generator, discriminator, dataloader)


[Epoch 0/5] [Batch 0/1] [D loss: 0.6934985518455505] [G loss: 0.7065462470054626]
[Epoch 1/5] [Batch 0/1] [D loss: 0.6899080276489258] [G loss: 0.6821267008781433]
[Epoch 2/5] [Batch 0/1] [D loss: 0.6868503093719482] [G loss: 0.6739291548728943]
[Epoch 3/5] [Batch 0/1] [D loss: 0.6832177639007568] [G loss: 0.6760920286178589]
[Epoch 4/5] [Batch 0/1] [D loss: 0.6786555647850037] [G loss: 0.6850621104240417]
